In [2]:
from importlib import reload

import numpy as np
import polars as pl
from plotly import graph_objects as go

import coolsearch.models as cmodel
import coolsearch.search as csearch
import coolsearch.utility_functions as util
import coolsearch.plotting as cplt

cplt.set_plotly_template()

reload(cmodel)
reload(csearch)
reload(util)

<module 'coolsearch.utility_functions' from '/home/marcu/projects/cool_search/coolsearch/utility_functions.py'>

In [3]:
util.get_grid(
    {"x": 3, "y": 2},
    {"x": (-1, 1), "y": (-2, 2)},
    dict.fromkeys(["x", "y"], "float"),
)

x,y
f64,f64
-1.0,-2.0
-1.0,2.0
0.0,-2.0
0.0,2.0
1.0,-2.0
1.0,2.0


In [ ]:
f = util.test_function_01

xx = np.linspace(-10, 10, 10000)
yy = f(xx)

idx_min_true = np.argmin(yy)
print("true min", xx[idx_min_true], yy[idx_min_true])

In [ ]:
reload(cmodel)

samples = pl.DataFrame({"x": np.linspace(-10, 10, 5)})
samples = samples.with_columns(
    pl.col("x").map_elements(f, return_dtype=pl.Float64).alias("f"),
)


model = cmodel.PolynomialModel(
    samples=samples,
    features=["x"],
    degree=2,
    target="f",
)
model.fit()

print(model.polynomial)
f_pred = model.predict()

f_pred_poly = model.polynomial(samples["x"])
print(f_pred_poly)

samples = samples.with_columns(f_pred=f_pred)
display(samples)
min_poly, poly_val = model.minimum_numeric(1000)
display(min_poly)

In [ ]:
fig = go.Figure(layout=dict(title="Polynomial model"))
fig.add_traces(
    [
        go.Scatter(x=samples["x"], y=samples["f"], mode="markers", name="input"),
        go.Scatter(
            x=poly_val["x"], y=poly_val["value"], mode="lines", name="fitted poly"
        ),
        go.Scatter(x=xx, y=yy, mode="lines", name="true"),
    ]
)


In [ ]:
model.residuals